In [1]:
from keras.models import Model#Model()输入张量放KL.Input()里
import keras.layers as KL
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
#两种方法，定义函数compile时再指定Loss
def custom_loss1(y_true,y_pred):
    return K.mean(K.abs(y_true-y_pred))
def custom_loss2(y_true,y_pred):
    return K.mean(K.square(y_true-y_pred))

In [3]:
input_tensor1 = KL.Input((32,32,3))
input_tensor2 = KL.Input((4,))
input_target = KL.Input((2,))

x = KL.BatchNormalization(axis=-1)(input_tensor1)
x = KL.Conv2D(16,(3,3),padding='same')(x)
x = KL.Activation('relu')(x)
x = KL.MaxPool2D(2)(x)
x = KL.Conv2D(32,(3,3),padding='same')(x)
x = KL.Activation('relu')(x)
x = KL.MaxPool2D(2)(x)
x = KL.Flatten()(x)
x = KL.Dense(32)(x)
out2 = KL.Dense(2)(x)#网络内生成y_true

y = KL.Dense(32)(input_tensor2)
out1 = KL.Dense(2)(y)
loss1 = KL.Lambda(lambda x:custom_loss1(*x),name="loss1")([out2,out1])
loss2 = KL.Lambda(lambda x:custom_loss1(*x),name='loss2')([input_target,out1])
#给model喂进输入输出
model = Model([input_tensor1,input_tensor2,input_target],[out1,out2,loss1,loss2])

In [4]:
#compile前可以通过名字取出layer可直接编译
loss_lay1 = model.get_layer('loss1').output
loss_lay2 = model.get_layer('loss2').output

model.add_loss(loss_lay1)
model.add_loss(loss_lay2)
#不用再回传loss，但有几个输出定义几个None
model.compile(optimizer='sgd',loss=[None,None,None,None])

In [29]:
def data_gen(num):
    for i in range(num):
        #所有的loss都在model里进行了（model.add_loss）,不需要传入外部label了
        yield [np.random.normal(1,1,(1,32,32,3)),np.random.normal(1,1,(1,4)), np.random.normal(1,1,(1,2))],[]

In [30]:
dataSet = data_gen(10000)

In [31]:
model.fit_generator(dataSet,steps_per_epoch=100,epochs=20)

Epoch 1/20
100/100 [==============================] - 3s 32ms/step - loss: 1.3671
Epoch 2/20
100/100 [==============================] - 0s 5ms/step - loss: 1.1219
Epoch 3/20
100/100 [==============================] - 1s 5ms/step - loss: 1.0377
Epoch 4/20
100/100 [==============================] - 1s 6ms/step - loss: 1.0276
Epoch 5/20
100/100 [==============================] - 0s 5ms/step - loss: 0.9778
Epoch 6/20
100/100 [==============================] - 1s 5ms/step - loss: 1.0473
Epoch 7/20
100/100 [==============================] - 1s 5ms/step - loss: 0.9132
Epoch 8/20
100/100 [==============================] - 1s 5ms/step - loss: 0.9418
Epoch 9/20
100/100 [==============================] - 1s 5ms/step - loss: 0.9022
Epoch 10/20
100/100 [==============================] - 1s 5ms/step - loss: 0.9627
Epoch 11/20
100/100 [==============================] - 1s 6ms/step - loss: 0.9468
Epoch 12/20
100/100 [==============================] - 1s 5ms/step - loss: 1.0355
Epoch 13/20
100/100 [===

In [5]:
import keras.engine as KE
#重写keras自带Layer的两个方法（输入输出）
KE.Layer.__call__
KE.Layer.compute_output_shape

KL.Lambda(func)(input)#另一个定义层的方法

NameError: name 'func' is not defined

In [4]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='model.png',show_shapes=True)

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [5]:
blocks = [3,6,4]
for i,num in enumerate(blocks):
    print(i,num)

0 3
1 6
2 4
